# Capstone Final Project

## Introduction

To find the most suitable neighbourhood to rent in Bangalore, India
One of my friend is moving from Pune to Bangalore for a new job. He needs help in deciding which neighbourhood to select for renting. The decision will be based on distance from his new office location, his choice of preferred venue categories he would like near his place of stay. Number of venues nearby should also affect his decision.
We would also look at the affordability of the neighbourhoods based on rent data.
In short, we would look at the following 3 factors
1.	Proximity to Office
2.	Nearby Facilities
3.	Affordability


In [2]:
pip install folium

     |████████████████████████████████| 93 kB 2.4 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Data

We'll first take user input of list of venue categories he would like to have near his place of staying.
Then keeping his office location static, call foursquare API to check all the neighbourhoods within 3-4km. For doing this we would need to scrape pin code and locality data from an external link and latitude and longitude data from geopy
https://www.mapsofindia.com/pincode/india/karnataka/bangalore
Find all the nearby venues and count them against the user defined categories. Use Euclidean distance for finding the best fit neighbourhoods.
I would also be using the rent data from the following URL, to take into account affordability of the neighbourhood. Avg. rent column from the link would be used to define the rent of the neighbourhood
https://www.makaan.com/price-trends/property-rates-for-rent-in-bangalore
From the already shortlisted neighbourhoods, I'll check which one has the least amount of rent.
Priority should be given to proximity (will be taken care by foursquare radius variable), facilities (will be taken care by best fit neighbourhoods), affordability in that order (will be taken care by the rent data)


## Methodology

#### 1.	Defined User Preferences

Define different parameters to capture user preferences, like office address, proximity threshold from office, users list of preferred categories, and the weight to be provided for each feature

#### 2.	Find location of office

Based on the office address, we would first mark the location on the map and would work on all localities taking office address as the center.

#### 3.	Find neighbourhoods close to office

Using external links, we'll first figure out the postal codes and localities in Bnagalore. We'll then use geopy to assign latitudes and longitudes to the localities. We'll then calculate the distance of each locality from the office and filter out those which doesnt satify the distance threshold criteria

#### 4.	Check out venues in those neighbourhoods

Using foursquare API we'll explore all the venues near the filtered localities

#### 5.	Rate venues according to User's Preference List

We'll mark all venues against users preferrence, if venue category is present in user's preference list we'll mark it as one and then aggregate all the records with respect to the locality to get rating for every locality 

#### 6.	Upload rent for different regions

Using external links, we'll create a table to store locality vs rent data for a 2-bhk flat.

#### 7.	Add a distance column and rent column and matching score with user venue preference in a single table

We'll then merge all three parameters in a single table based on the locality

#### 8.	Define a metric for ranking

We'll define a new metric for calculating final rating for each locality based on the different weightages as specified by the user.

#### 9.	Final Ranking Table

We'll sort the table and select the top candidates based on highest final rating

#### 10.	Visualize preffered location with final ranking data (radius of circle)

We'll then try to visialize the preffered localities on the map. Most preffered would have the highest radius marker.


## 1. Defined User Preferences

In [4]:
office_address ='murugeshpalaya, Bangalore'
Proximity_from_office = 6.5 #in km
User_preference = ['Pub','Indian Restaurant','American Restaurant','Badminton Court','Pizza Place','Lounge','Gym','Burrito Place','Brewery','Breakfast Spot','Clothing Store','Convenience Store','Department Store','Diner','Fast Food Restaurant','Hospital','Italian Restaurant','Liquor Store','Market','Mexican Restaurant','Middle Eastern Restaurant','Movie Theater','Multiplex','Nightclub','North Indian Restaurant','Park','Playground','Salad Place','Shopping Mall','Steakhouse']
User_preference_short=['Pub','Badminton Court','Pizza Place','Park','Gym','Brewery','Department Store','Hospital','Diner','Liquor Store','Salad Place']
Proximity_weight = 3
Facility_weight = 2
Affordibility_weight = 1

## 2. Find Office Location

In [5]:
geolocator = Nominatim(user_agent="foursquare_agent")
location_office = geolocator.geocode(office_address)
latitude_office = location_office.latitude
longitude_office = location_office.longitude
print('The geograpical coordinate of Office is {}, {}.'.format(latitude_office, longitude_office))

The geograpical coordinate of Office is 12.9561284, 77.6575713.


#### Visualize Office Location

In [6]:
import folium
# create map of New York using latitude and longitude values
map_bangalore_full = folium.Map(location=[latitude_office, longitude_office], zoom_start=12)

# add markers to map
label = 'Office Address, Murugeshpalaya'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude_office, longitude_office],
    radius=10,
    popup=label,
    color='green',
    fill=True,
    fill_color='green',
    fill_opacity=1,
    parse_html=False).add_to(map_bangalore_full)
    
map_bangalore_full

## 3. Find Neighbourhood close to office

#### 2.1 Upload all neighbourhoods of Bangalore with Postal Code

In [7]:

#mytable = soup.find_all('table')
import requests
url = requests.get('https://www.mapsofindia.com/pincode/india/karnataka/bangalore/').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())

mytable = soup.find('table',{'class':''})

In [8]:
#Create array to hold the data we extract
postal_code = []
borough = []
neighbourhood = []

#for table in mytable:
rows = mytable.find_all('tr')
    
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        #postal_code = cells[0]
        postal_code.append(cells[1].text.strip())

        #borough = cells[1]
        borough.append(cells[0].text.strip())

df = pd.DataFrame()
df['PostalCode'] = postal_code
df['Borough'] = borough
df.drop(df[df['PostalCode']=='Pincode'].index, inplace = True) 
df.head()

,PostalCode,Borough
1,560063,A F station yelahanka
2,560030,Adugodi
3,560034,Agara
4,560007,Agram
5,560007,Air Force hospital


### 3.2 Find latitude and longitude information

In [9]:
i=1
for postalcode in df['PostalCode']:
    address = 'Bangalore '+ str(postalcode)
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    if (location!=None):
        df.loc[i,'Latitude'] = location.latitude
        df.loc[i,'Longitude'] = location.longitude
        #print('The geograpical coordinate of Office is {}, {}.'.format(latitude, longitude))
    i=i+1

#### Visualize localities

In [10]:
import folium
# create map of New York using latitude and longitude values
map_bangalore_full = folium.Map(location=[latitude_office, longitude_office], zoom_start=12)

# add markers to map
for lat, lng, borough in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore_full)  

label = 'Office Address, Murugeshpalaya'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude_office, longitude_office],
    radius=10,
    popup=label,
    color='green',
    fill=True,
    fill_color='green',
    fill_opacity=1,
    parse_html=False).add_to(map_bangalore_full)
    
map_bangalore_full

### 3.3 Find distance from office

In [11]:
# Python 3 program to calculate Distance Between Two Points on Earth 
from math import radians, cos, sin, asin, sqrt 
def distance(lat1, lat2, lon1, lon2): 
      
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    return(c * r) 

In [12]:
for i, row in df.iterrows():
    df.loc[i,'Distance'] = distance(df.loc[i]['Latitude'],latitude_office,df.loc[i]['Longitude'],longitude_office)
df.head()

,PostalCode,Borough,Latitude,Longitude,Distance
1,560063,A F station yelahanka,12.898725,77.639322,6.682357
2,560030,Adugodi,12.941862,77.602382,6.187522
3,560034,Agara,12.928178,77.635982,3.890116
4,560007,Agram,12.968553,77.630313,3.260862
5,560007,Air Force hospital,12.968553,77.630313,3.260862


### 3.4 Dropping locations whose distance is greater than user defined criteria

In [13]:
import folium
# create map of New York using latitude and longitude values
map_bangalore_full = folium.Map(location=[latitude_office, longitude_office], zoom_start=12)

# add markers to map
for lat, lng, borough, dist in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Distance']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore_full) 
    label2 = '{}'.format(dist)+'km'
    label2 = folium.Popup(dist, parse_html=True)
    #folium.PolyLine(locations=[[latitude_office, longitude_office], [lat, lng]], color='blue',popup=dist,weight=.7, opacity=.7).add_to(map_bangalore_full)

label = 'Office Address, Murugeshpalaya'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude_office, longitude_office],
    radius=10,
    popup=label,
    color='green',
    fill=True,
    fill_color='green',
    fill_opacity=1,
    parse_html=False).add_to(map_bangalore_full)
    
folium.Circle(location=[latitude_office, longitude_office], popup='Point 1A', fill_color='#000', radius=1000 * Proximity_from_office, weight=2, color="#000").add_to(map_bangalore_full)

map_bangalore_full

In [43]:
bangalore_data=df[df['Distance']<Proximity_from_office].reset_index(drop=True)

#### After filtering

In [44]:
import folium
# create map of New York using latitude and longitude values
map_bangalore_full = folium.Map(location=[latitude_office, longitude_office], zoom_start=12)

# add markers to map
for lat, lng, borough, dist in zip(bangalore_data['Latitude'], bangalore_data['Longitude'], bangalore_data['Borough'], bangalore_data['Distance']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore_full) 
    label2 = '{}'.format(dist)+'km'
    label2 = folium.Popup(dist, parse_html=True)
    #folium.PolyLine(locations=[[latitude_office, longitude_office], [lat, lng]], color='blue',popup=dist,weight=.7, opacity=.7).add_to(map_bangalore_full)

label = 'Office Address, Murugeshpalaya'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude_office, longitude_office],
    radius=10,
    popup=label,
    color='green',
    fill=True,
    fill_color='green',
    fill_opacity=1,
    parse_html=False).add_to(map_bangalore_full)
    
folium.Circle(location=[latitude_office, longitude_office], popup='Point 1A', fill_color='#000', radius=1000 * Proximity_from_office, weight=2, color="#000").add_to(map_bangalore_full)

map_bangalore_full

## 4 Explore venues in those shorlisted neighbourhoods

In [45]:
CLIENT_ID = 'ONOOXEKNAJ1TGGGEJT0FQVF5WNPZVQH2I0JAA2ACPTE3K4ER' # your Foursquare ID
CLIENT_SECRET = 'HAJ3ODQXGK3I0LCU5KLCNGOVIBVFYSUXHUQ1XAY2L0GFUIEX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ONOOXEKNAJ1TGGGEJT0FQVF5WNPZVQH2I0JAA2ACPTE3K4ER
CLIENT_SECRET:HAJ3ODQXGK3I0LCU5KLCNGOVIBVFYSUXHUQ1XAY2L0GFUIEX


In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:

# type your answer here
bangalore_venues = getNearbyVenues(names=bangalore_data['Borough'],
                                   latitudes=bangalore_data['Latitude'],
                                   longitudes=bangalore_data['Longitude']
                                  )

Adugodi
Agara
Agram
Air Force hospital
Austin Town
Banaswadi
Bangalore Air port
Bangalore Bazaar
Bangalore Sub fgn post
Bangalore.
Basaveshwaranagar
Bellandur
Brigade Road
C.V.raman nagar
Cmp Centre and school
Cubban Road
Dharmaram College
Doddanekkundi
Domlur
Doorvaninagar
Dr. ambedkar veedhi
H.A.l ii stage
Hsr Layout
Highcourt
Hoodi
Horamavu
Hosur Road
Hulsur Bazaar
Immedihalli
Indiranagar
Indiranagar Com. complex
Jalavayuvihar
Jeevanbhimanagar
K H b colony
Kalyanagar
Kamakshipalya
Kendriya Sadan
Koramangala
Koramangala I block
Koramangala Vi bk
Krishnarajapuram R s
Kundalahalli
Legislators Home
Mahadevapura
Mahatma Gandhi road
Marathahalli Colony
Museum Road
Nal
Narayan Pillai street
New Thippasandra
Rajbhavan
Ramagondanahalli
Ramamurthy Nagar
Rameshnagar
Richmond Town
Sivan Chetty gardens
Someswarapura
St. john's medical college
Taverekere
Vidhana Soudha
Vimapura
Viveknagar
Whitefield
Yemalur


In [47]:
print(bangalore_venues.shape)
bangalore_venues.head()

(1274, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adugodi,12.941862,77.602382,Cafe Coffee Day,12.943541,77.600029,Café
1,Adugodi,12.941862,77.602382,Domino's Pizza,12.942000,77.598000,Pizza Place
2,Adugodi,12.941862,77.602382,Zapak Gameplex,12.942955,77.602910,Gaming Cafe
3,Adugodi,12.941862,77.602382,Mico grounds,12.943330,77.605132,Playground
4,Agara,12.928178,77.635982,Cafe Thulp,12.928999,77.635208,Burger Joint


## 5. Rate venues according to User's Preference List

In [48]:
bangalore_venues['rating']=0
for index,row in bangalore_venues.iterrows():
    if bangalore_venues.loc[index]['Venue Category'] in User_preference_short:
        #print("Yes, 'at' found in List : " , bangalore_venues.loc[index]['Venue Category'])
        bangalore_venues.loc[index,'rating']=1
        

In [49]:
bangalore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,rating
0,Adugodi,12.941862,77.602382,Cafe Coffee Day,12.943541,77.600029,Café,0
1,Adugodi,12.941862,77.602382,Domino's Pizza,12.942000,77.598000,Pizza Place,1
2,Adugodi,12.941862,77.602382,Zapak Gameplex,12.942955,77.602910,Gaming Cafe,0
3,Adugodi,12.941862,77.602382,Mico grounds,12.943330,77.605132,Playground,0
4,Agara,12.928178,77.635982,Cafe Thulp,12.928999,77.635208,Burger Joint,0


### 5.1 Sum across the localities

In [50]:
a = ['Neighborhood','rating']
choices_match = bangalore_venues[a]
choices_match = choices_match.groupby('Neighborhood').sum()   
choices_match

,rating
Neighborhood,
Adugodi,1
Agara,2
Agram,1
Air Force hospital,1
Austin Town,0
Banaswadi,0
Bangalore Air port,1
Bangalore Bazaar,4
Bangalore Sub fgn post,8


In [51]:
#Merge with distance column
distance_choice = pd.merge(bangalore_data, choices_match, how='left', left_on='Borough',right_on='Neighborhood')
distance_choice.head()

,PostalCode,Borough,Latitude,Longitude,Distance,rating
0,560030,Adugodi,12.941862,77.602382,6.187522,1.0
1,560034,Agara,12.928178,77.635982,3.890116,2.0
2,560007,Agram,12.968553,77.630313,3.260862,1.0
3,560007,Air Force hospital,12.968553,77.630313,3.260862,1.0
4,560047,Austin Town,12.966216,77.614254,4.826111,0.0


## 6. Upload rent for different regions

In [53]:
rent = []
borough = []

#mytable = soup.find_all('table')
import requests
url = requests.get('https://www.makaan.com/price-trends/property-rates-for-rent-in-bangalore').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())

mytable = soup.find('table',{'class':'tbl'})
#for table in mytable:
rows = mytable.find_all('tr')
    
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        #postal_code = cells[0]
        rent.append((cells[4].text.strip()).replace(',',''))
        #rent1.append(cells[4].text.strip())
        #borough = cells[1]
        borough.append(cells[0].text.strip())

for i in range(84):
    url = requests.get('https://www.makaan.com/price-trends/property-rates-for-rent-in-bangalore?page='+str(i+2)).text
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(url,'lxml')

    mytable = soup.find('table',{'class':'tbl'})
    #for table in mytable:
    rows = mytable.find_all('tr')

    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 1:
            #postal_code = cells[0]
            rent.append((cells[4].text.strip()).replace(',',''))
            #rent1.append(cells[4].text.strip())
            #borough = cells[1]
            borough.append(cells[0].text.strip())


In [54]:
#Create array to hold the data we extract
df1 = pd.DataFrame()
df1['Borough'] = borough
df1['Rent'] = rent
df1.drop(df1[df1['Rent']=='-'].index, inplace = True) 
df1

,Borough,Rent
0,HSR Layout,26700
1,Koramangala,29525.64
2,Whitefield,18346.65
3,Krishnarajapura,15176.47
4,Begur,14681.25
5,Varthur,14400
6,Devanahalli,15285.71
7,Chandapura,11466.67
8,Mahadevapura,17521.43
9,Whitefield Hope Farm Junction,16000


In [35]:
df1.sort_values('Borough')

,Borough,Rent
334,10 1 4th Cross Rd B T M La,12333.33
597,100 Feet Ring Road,28200
1219,10th Cross Road,23000
451,14th Main,16700
1222,15th Cross,27800
598,17th Main,25284.86
362,1st Block Jayanagar,22000
1224,1st Cross,10000
260,1st Phase JP Nagar,16500
798,1st stage,32000


In [56]:
df1.sort_values('Borough')

,Borough,Rent
334,10 1 4th Cross Rd B T M La,12333.33
597,100 Feet Ring Road,28200
1219,10th Cross Road,23000
451,14th Main,16700
1222,15th Cross,27800
598,17th Main,25284.86
362,1st Block Jayanagar,22000
1224,1st Cross,10000
260,1st Phase JP Nagar,16500
798,1st stage,32000


In [60]:
#Some modifications to match data with Pin Code Data (extra space modification, extra letter modification)
df1.loc[56,'Borough'] = 'C.V.raman nagar'
df1.loc[97,'Borough'] = 'Doddanekkundi'
df1.loc[424,'Borough'] = 'Doorvaninagar'
df1.loc[579,'Borough'] = 'H.A.l ii stage'
df1.loc[0,'Borough'] = 'Hsr Layout'
df1.loc[1512,'Borough'] = 'K H b colony'
df1.loc[704,'Borough'] = 'Koramangala Vi bk'
df1.loc[13,'Borough'] = 'Marathahalli Colony'
df1.loc[787,'Borough'] = 'Sivan Chetty gardens'
df1.loc[509,'Borough'] = 'Vimapura'
df1.loc[405,'Borough'] = 'Viveknagar'
df1.loc[3,'Borough'] = 'Krishnarajapuram R s'
df1.loc[912,'Borough'] = 'Brigade Road'
df1.loc[825,'Borough'] = 'Hosur Road'
df1.loc[459,'Borough'] = 'Immedihalli'
df1.loc[74,'Borough'] = 'Jeevanbhimanagar'
df1.loc[25,'Borough'] = 'Kalyanagar'
df1.loc[499,'Borough'] = 'Mahatma Gandhi road'
df1.loc[25,'Borough'] = 'Kalyanagar'
df1.loc[25,'Borough'] = 'Kalyanagar'

## 7. Add a distance column and rent column and matching score with user venue preference in a single table

In [61]:
distance_choice_rent = pd.merge(distance_choice, df1, how='left', left_on='Borough',right_on='Borough')
distance_choice_rent

,PostalCode,Borough,Latitude,Longitude,Distance,rating,Rent
0,560030,Adugodi,12.941862,77.602382,6.187522,1.0,35450
1,560034,Agara,12.928178,77.635982,3.890116,2.0,NaN
2,560007,Agram,12.968553,77.630313,3.260862,1.0,NaN
3,560007,Air Force hospital,12.968553,77.630313,3.260862,1.0,NaN
4,560047,Austin Town,12.966216,77.614254,4.826111,0.0,30000
5,560043,Banaswadi,13.012462,77.650918,6.305318,0.0,22178.95
6,560017,Bangalore Air port,12.955934,77.658272,0.078934,1.0,NaN
7,560001,Bangalore Bazaar,12.973013,77.603567,6.145695,4.0,NaN
8,560025,Bangalore Sub fgn post,12.968306,77.607173,5.626635,8.0,NaN
9,560001,Bangalore.,12.973013,77.603567,6.145695,4.0,NaN


In [62]:
#Subsitute NA Rating values with 0
distance_choice_rent['rating'] = distance_choice_rent['rating'].fillna(0)

#Subsitute NA Rent values with average rent?
distance_choice_rent.dropna(subset=["Rent"], inplace = True) 
#average_rent = distance_choice_rent.mean()
#distance_choice_rent['Rent'] = distance_choice_rent['Rent'].fillna(average_rent)

## 8. Define a metric for ranking

In [63]:
def normalize(df):
    normalized_df=(df-df.min())/df.std()
    return normalized_df

distance_choice_rent['Final Rating'] = normalize(-Proximity_weight * normalize(distance_choice_rent['Distance']) + Facility_weight * normalize(distance_choice_rent['rating']) + Affordibility_weight*normalize(distance_choice_rent['Rent'].astype(float)))

In [64]:
distance_choice_rent.sort_values(by=['Final Rating'], inplace=True, ascending=False)

## 9. Final Ranking Table

In [65]:
distance_choice_rent

,PostalCode,Borough,Latitude,Longitude,Distance,rating,Rent,Final Rating
29,560038,Indiranagar,12.978634,77.642143,3.009491,9.0,38333.33,4.223302
18,560071,Domlur,12.957436,77.640509,1.854605,3.0,44571.43,3.404427
21,560008,H.A.l ii stage,12.963734,77.645960,1.516027,4.0,18000,3.165068
60,560017,Vimapura,12.955934,77.658272,0.078934,1.0,12750,2.939794
54,560025,Richmond Town,12.968306,77.607173,5.626635,8.0,31000,2.625870
49,560075,New Thippasandra,12.971953,77.662304,1.832790,1.0,21500,2.363898
39,560095,Koramangala Vi bk,12.945245,77.619083,4.342902,3.0,39000,2.154898
37,560034,Koramangala,12.928178,77.635982,3.890116,2.0,29525.64,1.882305
44,560001,Mahatma Gandhi road,12.973013,77.603567,6.145695,4.0,50400,1.868557
45,560037,Marathahalli Colony,12.964719,77.677864,2.397479,0.0,20611.11,1.841589


## 10. Visualize preffered location with final ranking data (radius of circle)

In [66]:
import folium
# create map of New York using latitude and longitude values
map_bangalore_full = folium.Map(location=[latitude_office, longitude_office], zoom_start=13)

# add markers to map
for lat, lng, borough, rate in zip(distance_choice_rent['Latitude'], distance_choice_rent['Longitude'], distance_choice_rent['Borough'], distance_choice_rent['Final Rating']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=rate*4+1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore_full) 
    label2 = '{}'.format(dist)+'km'
    label2 = folium.Popup(dist, parse_html=True)
    #folium.PolyLine(locations=[[latitude_office, longitude_office], [lat, lng]], color='blue',popup=dist,weight=.7, opacity=.7).add_to(map_bangalore_full)

label = 'Office Address, Murugeshpalaya'
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude_office, longitude_office],
    radius=10,
    popup=label,
    color='green',
    fill=True,
    fill_color='green',
    fill_opacity=1,
    parse_html=False).add_to(map_bangalore_full)
    
#folium.Circle(location=[latitude_office, longitude_office], popup='Point 1A', fill_color='#000', radius=1000 * Proximity_from_office, weight=2, color="#000").add_to(map_bangalore_full)

map_bangalore_full